## Import

In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import DualTransform

import random
import time
import warnings
import sys
import argparse
import shutil

import torch.backends.cudnn as cudnn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import LambdaLR

import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torchvision import models
from collections import OrderedDict
from pytorch_model_summary import summary
from randaugment import RandAugment
from scipy import stats
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#config
#train_source, val_source: (2048,1024)
#train_target,test : (1920,1080)

#class 정보
# 0 : Road
# 1 : Sidewalk
# 2 : Construction
# 3 : Fence
# 4 : Pole 
# 5 : Traffic Light
# 6 : Traffic Sign
# 7 : Nature
# 8 : Sky
# 9 : Person
# 10 : Rider
# 11 : Car

batch_size = 4
epochs = 50
num_class = 13
source_root = './home/js/ssai/open/data/train_source_image'
target_root = './home/js/ssai/open/data/train_target_image'
learning_rate = 2.5e-3
iters_per_epoch = 2500
seed = 42
weight_decay = 0.0005
momentum = 0.9
train_size = (512,512)
test_output_size = (960,540)
lr_power = 0.9
lr_d =1e-4
ignore_label = 255
trade_off = 0.01 #0.001 

palette = [[0, 94, 135], [242, 255, 97], [165, 42, 42], [0, 0, 192],
                [197, 226, 255], [0, 60, 100], [0, 0, 142], [62, 200, 71],
                [255,207,157], [0, 187, 255], [255, 102, 163],[166,97,247],[0,0,0]]

palette = np.array(palette)

## Utils

In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Define Model

In [4]:
#deeplabv2


model_urls = {
    'deeplabv2_resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth'
}

#https://discuss.pytorch.org/t/how-to-load-load-state-dict-for-resnet-model/125998/2
affine_par = True

class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)  # change

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion or dilation == 2 or dilation == 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, affine=affine_par))
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)  # change
        self.bn1 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False

        padding = dilation

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,  # change
                               padding=padding, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn2.parameters():
            i.requires_grad = False

        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4, affine=affine_par)
        for i in self.bn3.parameters():
            i.requires_grad = False

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ASPP_V2(nn.Module):
    def __init__(self, inplanes, dilation_series, padding_series, num_classes):
        super(ASPP_V2, self).__init__()
        self.conv2d_list = nn.ModuleList()
        for dilation, padding in zip(dilation_series, padding_series):
            self.conv2d_list.append(
                nn.Conv2d(inplanes, num_classes, kernel_size=3, stride=1, padding=padding, dilation=dilation,
                          bias=True))
        for m in self.conv2d_list:
            m.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.conv2d_list[0](x)
        for i in range(len(self.conv2d_list) - 1):
            out += self.conv2d_list[i + 1](x)

        return out


class Deeplab(nn.Module):
    def __init__(self, backbone, classifier, num_classes):
        super(Deeplab, self).__init__()
        self.backbone = backbone
        self.classifier = classifier
        self.num_classes = num_classes

    def forward(self, x):
        x = self.backbone(x)
        y = self.classifier(x)
        return y

    def get_1x_lr_params_NOscale(self):
        """
        This generator returns all the parameters of the net except for
        the last classification layer. Note that for each batchnorm layer,
        requires_grad is set to False in deeplab_resnet.py, therefore this function does not return
        any batchnorm parameter
        """
        # layers = [self.backbone.conv1, self.backbone.bn1,
        #         self.backbone.layer1, self.backbone.layer2, self.backbone.layer3, self.backbone.layer4]
        layers = [list(self.backbone.children())[0], list(self.backbone.children())[1],
                list(self.backbone.children())[4], list(self.backbone.children())[5], 
                list(self.backbone.children())[6], list(self.backbone.children())[7]]
        for layer in layers:
            for module in layer.modules():
                for param in module.parameters():
                    if param.requires_grad:
                        yield param

    def get_10x_lr_params(self):
        """
        This generator returns all the parameters for the last layer of the net,
        which does the classification of pixel into classes
        """
        for param in self.classifier.parameters():
            yield param

    def get_parameters(self, lr=1.):
        return [
            {'params': self.get_1x_lr_params_NOscale(), 'lr': 0.1 * lr},
            {'params': self.get_10x_lr_params(), 'lr': lr}
        ]

def deeplabv2_resnet101(num_classes=13, pretrained_backbone=True):
    """Constructs a DeepLabV2 model with a ResNet-1 01 backbone.

     Args:
         num_classes (int, optional): number of classes. Default: 19
         pretrained_backbone (bool, optional): If True, returns a model pre-trained on ImageNet. Default: True.
     """
    backbone = ResNet(Bottleneck, [3, 4, 23, 3])
    if pretrained_backbone:
        # download from Internet
        #saved_state_dict = load_state_dict_from_url(model_urls['deeplabv2_resnet101'], map_location=lambda storage, loc: storage, file_name="deeplabv2_resnet101.pth")
        #pre_backbone= models.resnet101(pretrained=True)
        #pre_backbone = nn.Sequential(*list(backbone.children())[:-2])
        ckpt = torch.load('./cityscapes_epoch_59.pth')
        
        # state_dict = ckpt['model']
        new_state_dict = OrderedDict()
        for k, v in ckpt.items():
            if not k.split('.')[1] == 'layer5':
                name =  k.replace("Scale.","")
                new_state_dict[name] = v
            
        backbone.load_state_dict(new_state_dict)
        #new_params = backbone.state_dict().copy()
        # for i in saved_state_dict:
        #     i_parts = i.split('.')
        #     if not i_parts[1] == 'layer5':
        #         new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
        # backbone.load_state_dict(new_params)

    classifier = ASPP_V2(2048, [6, 12, 18, 24], [6, 12, 18, 24], num_classes)
    return Deeplab(backbone, classifier, num_classes)


In [5]:
from typing import Optional, List


class AverageMeter(object):
    r"""Computes and stores the average and current value.

    Examples::

        >>> # Initialize a meter to record loss
        >>> losses = AverageMeter()
        >>> # Update meter after every minibatch update
        >>> losses.update(loss_value, batch_size)
    """
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class AverageMeterDict(object):
    def __init__(self, names: List, fmt: Optional[str] = ':f'):
        self.dict = {
            name: AverageMeter(name, fmt) for name in names
        }

    def reset(self):
        for meter in self.dict.values():
            meter.reset()

    def update(self, accuracies, n=1):
        for name, acc in accuracies.items():
            self.dict[name].update(acc, n)

    def average(self):
        return {
            name: meter.avg for name, meter in self.dict.items()
        }

    def __getitem__(self, item):
        return self.dict[item]


class Meter(object):
    """Computes and stores the current value."""
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0

    def update(self, val):
        self.val = val

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        #print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [6]:
import torch
import prettytable

__all__ = ['keypoint_detection']

def binary_accuracy(output: torch.Tensor, target: torch.Tensor) -> float:
    """Computes the accuracy for binary classification"""
    with torch.no_grad():
        batch_size = target.size(0)
        pred = (output >= 0.5).float().t().view(-1)
        correct = pred.eq(target.view(-1)).float().sum()
        correct.mul_(100. / batch_size)
        return correct


def accuracy(output, target, topk=(1,)):
    r"""
    Computes the accuracy over the k top predictions for the specified values of k

    Args:
        output (tensor): Classification outputs, :math:`(N, C)` where `C = number of classes`
        target (tensor): :math:`(N)` where each value is :math:`0 \leq \text{targets}[i] \leq C-1`
        topk (sequence[int]): A list of top-N number.

    Returns:
        Top-N accuracies (N :math:`\in` topK).
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target[None])

        res = []
        for k in topk:
            correct_k = correct[:k].flatten().sum(dtype=torch.float32)
            res.append(correct_k * (100.0 / batch_size))
        return res


class ConfusionMatrix(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.mat = None

    def update(self, target, output):
        """
        Update confusion matrix.

        Args:
            target: ground truth
            output: predictions of models

        Shape:
            - target: :math:`(minibatch, C)` where C means the number of classes.
            - output: :math:`(minibatch, C)` where C means the number of classes.
        """
        n = self.num_classes
        if self.mat is None:
            self.mat = torch.zeros((n, n), dtype=torch.int64, device=target.device)
        with torch.no_grad():
            k = (target >= 0) & (target < n)
            inds = n * target[k].to(torch.int64) + output[k]
            self.mat += torch.bincount(inds, minlength=n**2).reshape(n, n)

    def reset(self):
        self.mat.zero_()

    def compute(self):
        """compute global accuracy, per-class accuracy and per-class IoU"""
        h = self.mat.float()
        acc_global = torch.diag(h).sum() / h.sum()
        acc = torch.diag(h) / h.sum(1)
        iu = torch.diag(h) / (h.sum(1) + h.sum(0) - torch.diag(h))
        return acc_global, acc, iu

    # def reduce_from_all_processes(self):
    #     if not torch.distributed.is_available():
    #         return
    #     if not torch.distributed.is_initialized():
    #         return
    #     torch.distributed.barrier()
    #     torch.distributed.all_reduce(self.mat)

    def __str__(self):
        acc_global, acc, iu = self.compute()
        return (
            'global correct: {:.1f}\n'
            'average row correct: {}\n'
            'IoU: {}\n'
            'mean IoU: {:.1f}').format(
                acc_global.item() * 100,
                ['{:.1f}'.format(i) for i in (acc * 100).tolist()],
                ['{:.1f}'.format(i) for i in (iu * 100).tolist()],
                iu.mean().item() * 100)

    def format(self, classes: list):
        """Get the accuracy and IoU for each class in the table format"""
        acc_global, acc, iu = self.compute()

        table = prettytable.PrettyTable(["class", "acc", "iou"])
        for i, class_name, per_acc, per_iu in zip(range(len(classes)), classes, (acc * 100).tolist(), (iu * 100).tolist()):
            table.add_row([class_name, per_acc, per_iu])

        return 'global correct: {:.1f}\nmean correct:{:.1f}\nmean IoU: {:.1f}\n{}'.format(
            acc_global.item() * 100, acc.mean().item() * 100, iu.mean().item() * 100, table.get_string())



In [7]:
def send_to_device(tensor, device):
    """
    Recursively sends the elements in a nested list/tuple/dictionary of tensors to a given device.

    Args:
        tensor (nested list/tuple/dictionary of :obj:`torch.Tensor`):
            The data to send to a given device.
        device (:obj:`torch.device`):
            The device to send the data to

    Returns:
        The same data structure as :obj:`tensor` with all tensors sent to the proper device.
    """
    if isinstance(tensor, (list, tuple)):
        return type(tensor)(send_to_device(t, device) for t in tensor)
    elif isinstance(tensor, dict):
        return type(tensor)({k: send_to_device(v, device) for k, v in tensor.items()})
    elif not hasattr(tensor, "to"):
        return tensor
    return tensor.to(device)


class ForeverDataIterator:
    r"""A data iterator that will never stop producing data"""

    def __init__(self, data_loader: DataLoader, device=None):
        self.data_loader = data_loader
        self.iter = iter(self.data_loader)
        self.device = device

    def __next__(self):
        try:
            data = next(self.iter)
            if self.device is not None:
                data = send_to_device(data, self.device)
        except StopIteration:
            self.iter = iter(self.data_loader)
            data = next(self.iter)
            if self.device is not None:
                data = send_to_device(data, self.device)
        return data

    def __len__(self):
        return len(self.data_loader)

## Model Train

In [8]:
#train

def train(train_source_iter: ForeverDataIterator, train_target_iter: ForeverDataIterator,
          model, interp, criterion, dann,
          optimizer: SGD, lr_scheduler: LambdaLR, optimizer_d: SGD, lr_scheduler_d: LambdaLR,
          epoch: int):
    
    losses_s = AverageMeter('Loss (s)', ':3.2f')
    losses_transfer = AverageMeter('Loss (transfer)', ':3.2f')
    losses_discriminator = AverageMeter('Loss (discriminator)', ':3.2f')
    accuracies_s = Meter('Acc (s)', ':3.2f')
    accuracies_t = Meter('Acc (t)', ':3.2f')
    iou_s = Meter('IoU (s)', ':3.2f')
    iou_t = Meter('IoU (t)', ':3.2f')

    confmat_s = ConfusionMatrix(num_class)
    #confmat_t = ConfusionMatrix(model.num_classes)

    model.train()
    
    for i in range(iters_per_epoch):
        if(i % 100 == 0):
            print("iters : ",i)
        iter_start = time.time()
        x_s, label_s = next(train_source_iter)
        x_t = next(train_target_iter)

        x_s = x_s.to(device)
        label_s = label_s.long().to(device)
        x_t = x_t.to(device)
        #label_t = label_t.long().to(device)

        optimizer.zero_grad()
        optimizer_d.zero_grad()

        # Step 1: Train the segmentation network, freeze the discriminator
        dann.eval() #DomainAdversarialEntropyLoss
        y_s = model(x_s)
        pred_s = interp(y_s)
        #segmentaiton : cross-entropy loss (generator 즉 segmentataion network 학습)
        loss_cls_s = criterion(pred_s, label_s.squeeze(1))
        loss_cls_s.backward()

        # adversarial training to fool the discriminator
        y_t = model(x_t) # 똑같이 segmentation network에 target image를 넣고
        pred_t = interp(y_t)
        loss_transfer = dann(pred_t, 'source') # target image의 prediction 값을
        (loss_transfer * trade_off).backward()

        # Step 2: Train the discriminator
        dann.train()
        loss_discriminator = 0.5 * (dann(pred_s.detach(), 'source') + dann(pred_t.detach(), 'target'))
        loss_discriminator.backward()

        # compute gradient and do SGD step
        optimizer.step()
        optimizer_d.step()
        lr_scheduler.step()
        lr_scheduler_d.step()

        # measure accuracy and record loss
        losses_s.update(loss_cls_s.item(), x_s.size(0))
        losses_transfer.update(loss_transfer.item(), x_s.size(0))
        losses_discriminator.update(loss_discriminator.item(), x_s.size(0))

        confmat_s.update(label_s.flatten(), pred_s.argmax(1).flatten())
        #confmat_t.update(label_t.flatten(), pred_t.argmax(1).flatten())
        acc_global_s, acc_s, iu_s = confmat_s.compute()
        #acc_global_t, acc_t, iu_t = confmat_t.compute()
        accuracies_s.update(acc_s.mean().item())
        #accuracies_t.update(acc_t.mean().item())
        iou_s.update(iu_s.mean().item())

        iter_end = time.time()
        if(i % 100 == 0):
            print("seg_loss : ", loss_cls_s.item())
            print("entropy_loss: ",loss_transfer.item())
            print("disc_loss : ", loss_discriminator.item())
            print("iter end, time : ", iter_end-iter_start)
        #iou_t.update(iu_t.mean().item())

        # if i % args.print_freq == 0:
        #     progress.display(i)

        #     if visualize is not None:
        #         visualize(x_s[0], pred_s[0], label_s[0], "source_{}".format(i))
        #         visualize(x_t[0], pred_t[0], label_t[0], "target_{}".format(i))

def validate(val_loader: DataLoader, model, interp, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    acc = Meter('Acc', ':3.2f')
    iou = Meter('IoU', ':3.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, acc, iou],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    confmat = ConfusionMatrix(num_class)

    with torch.no_grad():
        for i, (x, label) in enumerate(val_loader):
            x = x.to(device)
            label = label.long().to(device)

            # compute output
            output = interp(model(x))
            loss = criterion(output, label)

            # measure accuracy and record loss
            losses.update(loss.item(), x.size(0))
            confmat.update(label.flatten(), output.argmax(1).flatten())
            acc_global, accs, iu = confmat.compute()
            acc.update(accs.mean().item())
            iou.update(iu.mean().item())

            # if i % args.print_freq == 0:
            #     progress.display(i)

            #     if visualize is not None:
            #         visualize(x[0], output[0], label[0], "val_{}".format(i))

    return confmat

In [9]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, source=True):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.source = source
        self.classes = ['Road','Sidewalk','Construction','Fence','Pole','Traffic Light'
                        ,'Traffic Sign','Nature','Sky','Person','Rider','Car','Background']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        if self.source==True:
            mask_path = self.data.iloc[idx, 2]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

            if self.transform:
            
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

                #transform 확인용
                
                #mask = palette[mask]
                # name = img_path.split('/')[-1]
                # cv2.imwrite(f'./trans/{name}.png', image)
                # cv2.imwrite(f'./trans/mask_{name}.png', mask)
            return image, mask
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image

        
    

In [10]:

scale = 5.5
def fisheye_distortion_rm(image):
    
    h,w = image.shape[:2]

    focal_length = w / 4
    center_x = w / 2 
    center_y = h / 2
    camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

    dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)

    map_x, map_y = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, None, (w, h), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    #undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
    undistorted_image = undistorted_image[int(h/scale):int(h-h/scale),int(w/scale):int(w-w/scale)]

    return undistorted_image

def fisheye_distortion_mask_rm(image):
    #image[image==0] = 255
    h,w = image.shape[:2]

    focal_length = w / 4
    center_x = w / 2 
    center_y = h / 2
    camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

    dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)

    map_x, map_y = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, None, (w, h), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT,borderValue=12)
    #undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
    undistorted_image = undistorted_image[int(h/scale):int(h-h/scale),int(w/scale):int(w-w/scale)].astype(np.uint8)
    #undistorted_image = np.round(undistorted_image).astype(np.uint8)
    #undistorted_image[undistorted_image==0] = 12
    #undistorted_image = filter(undistorted_image)
    #undistorted_image[undistorted_image>12] = 0

    return undistorted_image

class Fisheye(DualTransform):
    def __init__(self):
        super(Fisheye,self).__init__()
    
    def apply(self, img, **params):
        return fisheye_distortion_rm(img)

    def apply_to_mask(self, mask, **params):
        return fisheye_distortion_mask_rm(mask)
    
# def fisheye_distortion(image):
    
#     h,w = image.shape[:2]

#     focal_length = w / 4
#     center_x = w / 2 
#     center_y = h / 2
#     camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

#     dist_coeffs = np.array([0,0.5,0,0],dtype=np.uint8)

#     undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
#     #undistorted_image = undistorted_image[h//5:h-h//5,w//5:w-w//5]

#     return undistorted_image

# def fisheye_distortion_mask(image):
#     image[image==0] = 255
#     h,w = image.shape[:2]

#     focal_length = w / 4
#     center_x = w / 2 
#     center_y = h / 2
#     camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

#     dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)

#     undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
#     #undistorted_image = undistorted_image[h//5:h-h//5,w//5:w-w//5].astype(np.uint8)
#     #undistorted_image = np.round(undistorted_image).astype(np.uint8)
#     undistorted_image[undistorted_image==0] = 12
#     #undistorted_image = filter(undistorted_image)
#     undistorted_image[undistorted_image>12] = 0

#     return undistorted_image
   


In [11]:
def prob_2_entropy(prob):
    """ convert probabilistic prediction maps to weighted self-information maps
    """
    n, c, h, w = prob.size()
    return -torch.mul(prob, torch.log2(prob + 1e-30)) / np.log2(c)


def bce_loss(y_pred, y_label):
    y_truth_tensor = torch.FloatTensor(y_pred.size())
    y_truth_tensor.fill_(y_label)
    y_truth_tensor = y_truth_tensor.to(y_pred.get_device())
    return F.binary_cross_entropy_with_logits(y_pred, y_truth_tensor)

class Discriminator(nn.Sequential):
    """
    Domain discriminator model from
    `ADVENT: Adversarial Entropy Minimization for Domain Adaptation in Semantic Segmentation (CVPR 2019) <https://arxiv.org/abs/1811.12833>`_

    Distinguish pixel-by-pixel whether the input predictions come from the source domain or the target domain.
    The source domain label is 1 and the target domain label is 0.

    Args:
        num_classes (int): num of classes in the predictions
        ndf (int): dimension of the hidden features

    Shape:
        - Inputs: :math:`(minibatch, C, H, W)` where :math:`C` is the number of classes
        - Outputs: :math:`(minibatch, 1, H, W)`
    """
    def __init__(self, num_classes, ndf=64):
        super(Discriminator, self).__init__(
            nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=2, padding=1),
        )

class DomainAdversarialEntropyLoss(nn.Module):
    r"""The `Domain Adversarial Entropy Loss <https://arxiv.org/abs/1811.12833>`_

    Minimizing entropy with adversarial learning through training a domain discriminator.

    Args:
        domain_discriminator (torch.nn.Module): A domain discriminator object, which predicts
          the domains of predictions. Its input shape is :math:`(minibatch, C, H, W)` and output shape is :math:`(minibatch, 1, H, W)`

    Inputs:
        - logits (tensor): logits output of segmentation model
        - domain_label (str, optional): whether the data comes from source or target.
          Choices: ['source', 'target']. Default: 'source'

    Shape:
        - logits: :math:`(minibatch, C, H, W)` where :math:`C` means the number of classes
        - Outputs: scalar.

    Examples::

        >>> B, C, H, W = 2, 19, 512, 512
        >>> discriminator = Discriminator(num_classes=C)
        >>> dann = DomainAdversarialEntropyLoss(discriminator)
        >>> # logits output on source domain and target domain
        >>> y_s, y_t = torch.randn(B, C, H, W), torch.randn(B, C, H, W)
        >>> loss = 0.5 * (dann(y_s, "source") + dann(y_t, "target"))
    """
    def __init__(self, discriminator: nn.Module):
        super(DomainAdversarialEntropyLoss, self).__init__()
        self.discriminator = discriminator
        self.entropy = None

    def forward(self, logits, domain_label='source'):
        """
        """
        assert domain_label in ['source', 'target']
        probability = F.softmax(logits, dim=1) 
        entropy = prob_2_entropy(probability)
        self.entropy = entropy
        domain_prediciton = self.discriminator(entropy)
        if domain_label == 'source':
            return bce_loss(domain_prediciton, 1)
        else:
            return bce_loss(domain_prediciton, 0)

    def train(self, mode=True):
        r"""Sets the discriminator in training mode. In the training mode,
        all the parameters in discriminator will be set requires_grad=True.

        Args:
            mode (bool): whether to set training mode (``True``) or evaluation mode (``False``). Default: ``True``.
        """
        self.discriminator.train(mode)
        for param in self.discriminator.parameters():
            param.requires_grad = mode
        return self

    def eval(self):
        r"""Sets the module in evaluation mode. In the training mode,
        all the parameters in discriminator will be set requires_grad=False.

        This is equivalent with :meth:`self.train(False) <torch.nn.Module.train>`.
        """
        return self.train(False)

In [ ]:
class MaxSquareloss(nn.Module):
    def __init__(self, ignore_index= -1, num_class=13):
        super().__init__()
        self.ignore_index = ignore_index
        self.num_class = num_class
    
    def forward(self, pred, prob):
        """
        :param pred: predictions (N, C, H, W)
        :param prob: probability of pred (N, C, H, W)
        :return: maximum squares loss
        """
        # prob -= 0.5
        mask = (prob != self.ignore_index)    
        loss = -torch.mean(torch.pow(prob, 2)[mask]) / 2
        return loss
    

In [12]:
def main(resume=None):
    cudnn.benchmark = True

    transform = A.Compose(
    [   
        Fisheye(),
        A.Resize(train_size[1], train_size[0]),
        A.augmentations.geometric.transforms.HorizontalFlip(p=0.5),
        #A.augmentations.geometric.transforms.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()
    ])
    # Data loading code
    source_dataset = CustomDataset(csv_file='./train_source.csv', transform=transform, source=True)

    train_source_loader = DataLoader(source_dataset, batch_size=batch_size,
                                     shuffle=True, num_workers=1, pin_memory=True, drop_last=True)

    target_dataset = CustomDataset(csv_file='./train_target.csv', transform=transform, source=False)
    train_target_loader = DataLoader(target_dataset, batch_size=batch_size,
                                     shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
    
    val_target_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform)
    val_target_loader = DataLoader(val_target_dataset, batch_size=1, shuffle=False, pin_memory=True)

    train_source_iter = ForeverDataIterator(train_source_loader)
    train_target_iter = ForeverDataIterator(train_target_loader)

    # create model
    num_classes = num_class
    model = deeplabv2_resnet101(num_classes=num_classes).to(device)
    #model = deeplabv3(num_classes).to(device)
    discriminator = Discriminator(num_classes=num_classes).to(device)

    # define optimizer and lr scheduler
    optimizer = SGD(model.get_parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    optimizer_d = Adam(discriminator.parameters(), lr=lr_d, betas=(0.9, 0.99))
    lr_scheduler = LambdaLR(optimizer, lambda x: learning_rate * (1. - float(x) / epochs / iters_per_epoch) ** (lr_power))
    lr_scheduler_d = LambdaLR(optimizer_d, lambda x: (1. - float(x) / epochs / iters_per_epoch) ** (lr_power))

    #optionally resume from a checkpoint
    if resume:
        checkpoint = torch.load(resume, map_location='cpu')
        model.load_state_dict(checkpoint['model'])
        discriminator.load_state_dict(checkpoint['discriminator'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        optimizer_d.load_state_dict(checkpoint['optimizer_d'])
        lr_scheduler_d.load_state_dict(checkpoint['lr_scheduler_d'])
        start_epoch = checkpoint['epoch'] + 1
    else:
        start_epoch =0
    # define loss function (criterion)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=ignore_label).to(device) # segmentation loss
    dann = DomainAdversarialEntropyLoss(discriminator) # adversarial loss 
    interp_train = nn.Upsample(size=train_size[::-1], mode='bilinear', align_corners=True)
    interp_val = nn.Upsample(size=test_output_size[::-1], mode='bilinear', align_corners=True)


    # define visualization function
    #decode = source_dataset.decode_target

    # def visualize(image, pred, label, prefix):
    #     """
    #     Args:
    #         image (tensor): 3 x H x W
    #         pred (tensor): C x H x W
    #         label (tensor): H x W
    #         prefix: prefix of the saving image
    #     """
    #     image = image.detach().cpu().numpy()
    #     pred = pred.detach().max(dim=0)[1].cpu().numpy()
    #     label = label.cpu().numpy()
    #     for tensor, name in [TypeError: 'int' object is not callable
    #         (Image.fromarray(np.uint8(DeNormalizeAndTranspose()(image))), "image"),
    #         (decode(label), "label"),
    #         (decode(pred), "pred")
    #     ]:
    #         tensor.save(logger.get_image_path("{}_{}.png".format(prefix, name)))

    # if args.phase == 'test':
    #     confmat = validate(val_target_loader, model, interp_val, criterion, visualize, args)
    #     print(confmat)
    #     return

    # start training
    best_iou = 0.
    for epoch in range(start_epoch, epochs):
        print("current epoch :", epoch)
        print(lr_scheduler.get_lr(), lr_scheduler_d.get_lr())
        # train for one epoch
        print("train start")
        train_start = time.time()
        train(train_source_iter, train_target_iter, model, interp_train, criterion, dann, optimizer,
              lr_scheduler, optimizer_d, lr_scheduler_d, epoch)
        train_end = time.time()
        print("train end")
        print("epoch_time : ", train_end-train_start)
        # evaluate on validation set
        confmat = validate(val_target_loader, model, interp_train, criterion)
        print(confmat.format(source_dataset.classes))
        acc_global, acc, iu = confmat.compute()

        # calculate the mean iou over partial classes
        indexes = [source_dataset.classes.index(name) for name
                   in source_dataset.classes]
        iu = iu[indexes]
        mean_iou = iu.mean()

        #remember best acc@1 and save checkpoint
        if mean_iou > best_iou:
            torch.save(
            {
                'model': model.state_dict(),
                'discriminator': discriminator.state_dict(),
                'optimizer': optimizer.state_dict(),
                'optimizer_d': optimizer_d.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'lr_scheduler_d': lr_scheduler_d.state_dict(),
                'epoch': epoch,
            }, f'./save_path/model_save_v2_2500_{epoch}.pth' )
        
    
            #shutil.copy(logger.get_checkpoint_path(epoch), logger.get_checkpoint_path('best'))
        best_iou = max(best_iou, mean_iou)
        print("Target: {} Best: {}".format(mean_iou, best_iou))


## Run

In [13]:
#re_checkpoint = './save_path/model_save_v2_rmfishcr_rambda0.1_16.pth'
#main(re_checkpoint)
main()

/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torch/optim/sgd.py:27: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)
/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


current epoch : 0
[0.00025, 0.0025] [0.0001]
train start
iters :  0
seg_loss :  5.475464820861816
entropy_loss:  0.6969258189201355
disc_loss :  0.6931734085083008
iter end, time :  9.427624225616455
iters :  100
seg_loss :  0.44572100043296814
entropy_loss:  1.800594449043274
disc_loss :  0.5731470584869385
iter end, time :  1.6608281135559082
iters :  200
seg_loss :  0.3841615319252014
entropy_loss:  1.5036524534225464
disc_loss :  0.5719675421714783
iter end, time :  1.7051217555999756
iters :  300
seg_loss :  0.2780638337135315
entropy_loss:  1.4823514223098755
disc_loss :  0.5282875299453735
iter end, time :  1.6827328205108643
iters :  400
seg_loss :  0.2625495493412018
entropy_loss:  1.3714425563812256
disc_loss :  0.6585930585861206
iter end, time :  1.6478550434112549
iters :  500
seg_loss :  0.2537231147289276
entropy_loss:  1.1176496744155884
disc_loss :  0.5983473062515259
iter end, time :  1.6210060119628906
iters :  600
seg_loss :  0.15926018357276917
entropy_loss:  1.128

KeyboardInterrupt: 

##                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Entropy Map Visualization

In [ ]:
# target_layers = [model.model.blocks[-1].norm1]

# def reshape_transform(tensor, height=16, width=16):
#     result = tensor[:, 1 :  , :].reshape(tensor.size(0),
#         height, width, tensor.size(2))

#     # Bring the channels to the first dimension,
#     # like in CNNs.
#     result = result.transpose(2, 3).transpose(1, 2)
#     return result

# cam = GradCAM(model=model2.model, target_layers=target_layers, reshape_transform=reshape_transform)

# def analysis_image(image, target_class):
#   img = cv2.resize(image, [224, 224])
#   img = np.expand_dims(img, axis = 0)
#   img = torch.FloatTensor(img).permute(0,3,1,2)
#   input_tensor = img.cuda()

#   targets = [ClassifierOutputTarget(target_class)]

#   Cam = cam(input_tensor = input_tensor, targets = targets)

#   img2 = cv2.resize(image, [224, 224]) / 255.

#   Cam = np.expand_dims(np.squeeze(Cam, axis = 0), axis = -1)

#   visualization = show_cam_on_image(img2.astype(np.float32), Cam, use_rgb = True)

#   # colab에서 실행하기 때문에 cv2.imshow 대신 cv2_imshow 함수 이용
#   cv2_imshow(visualization)

## Inference

In [22]:
mode = 'val'
transform = A.Compose(
    [   
        A.Resize(train_size[0], train_size[1]),
        A.Normalize(),
        ToTensorV2()
    ])
if mode == 'val':
    test_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform, infer=True)
elif mode == 'test':
    test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)#
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=1)

In [20]:
def inference(save_model):
    interp_val = nn.Upsample(size=test_output_size[::-1], mode='bilinear', align_corners=True)
    with torch.no_grad():
        num_classes = num_class
        model = deeplabv2_resnet101(num_classes=num_classes).to(device)
        model.load_state_dict(save_model)
        model.eval()
        result = []
        iter=0
        batch_size=16
        for images in tqdm(test_dataloader):
            images = images.float().to(device)
            outputs = model(images)
            outputs = interp_val(outputs)
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()
            # batch에 존재하는 각 이미지에 대해서 반복
            for i,pred in enumerate(outputs,1):
                mask_img = np.zeros((540,960,3),dtype=np.uint8)
                pred = pred.astype(np.uint8)
                pred = Image.fromarray(pred) # 이미지로 변환
                pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
                pred = np.array(pred) # 다시 수치로 변환
                # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
                for class_id in range(12):
                    class_mask = (pred == class_id).astype(np.uint8)
                    mask_img = palette[pred]
                    # if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    #     mask_rle = rle_encode(class_mask)
                    #     result.append(mask_rle)
                    # else: # 마스크가 존재하지 않는 경우 -1
                    #     result.append(-1)

                if mode == 'val':
                    test = pd.read_csv('./val_source.csv')
                elif mode == 'test':
                    test = pd.read_csv('./test.csv')
                img_name=test['id'][iter*16+i-1]
                img_org = cv2.imread(test['img_path'][iter*16+i-1])
                img_org = cv2.resize(img_org,(960,540))
            
                result = np.hstack((img_org,mask_img))
                if mode == 'val':
                    new_gt = np.zeros((540,960,3),dtype=np.uint8)
                    img_gt = cv2.imread(test['gt_path'][iter*16+i-1],0)
                    img_gt = cv2.resize(img_gt,(960,540))
                    mask_255 = np.where(img_gt>=12)
                    img_gt[mask_255] = 12
                    new_gt = palette[img_gt]
                    result = np.hstack((img_org,mask_img,new_gt))
                    if not os.path.exists('./mask_save_s'):
                        os.makedirs('./mask_save_s')
                    cv2.imwrite(f"./mask_save_s/{img_name}.png",result)
                elif mode == 'test':    
                    if not os.path.exists('./mask_save'):
                        os.makedirs('./mask_save')
                    cv2.imwrite(f"./mask_save/{img_name}.png",result)

            iter+=1

    return result


In [23]:
model_path = './save_path/model_save_v2_2500_36.pth'
checkpoint = torch.load(model_path, map_location='cpu')
result = inference(checkpoint['model'])

  7%|▋         | 2/30 [00:10<02:20,  5.02s/it]


KeyboardInterrupt: 

## Submission

In [18]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,245291 13 245317 30 246241 74 247170 110 24810...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,1 79 602 438 1561 440 2520 441 3480 441 4439 4...
3,TEST_0000_class_3,-1
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,141119 2 142077 4 143036 5 143994 7 144953 8 1...
22772,TEST_1897_class_8,82 559 676 127 1043 557 1638 124 2004 555 2599...
22773,TEST_1897_class_9,227399 3 228359 3 229319 3 230279 3 231240 2 2...
22774,TEST_1897_class_10,-1


In [19]:
submit.to_csv('./rmfish2500_submit.csv', index=False)